In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

In [2]:
df = pd.read_csv('E:/Data Science/Data Sets/TikTok - Google Play Store/tiktok_google_play_reviews.csv')

In [3]:
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,gp:AOqpTOHRz-11c0apHLSKHHp52FxUXsQS9Z88wP3sWc5...,MR LOL GAMER,https://play-lh.googleusercontent.com/a/AATXAJ...,Good,5,0,23.8.4,2022-04-05 23:18:30,NaN,NaN
1,gp:AOqpTOF6mFDEkIypmyT3shDLjPHg8zB3kdns2W36ahp...,Dino Kljako,https://play-lh.googleusercontent.com/a-/AOh14...,Awesome app! Too many people on it where it's ...,5,0,NaN,2022-04-05 23:18:21,NaN,NaN
2,gp:AOqpTOGtqU4sb8vuVo3-eB7kIXWoBn-0YCUZ1SnPRKS...,Olivia Harding,https://play-lh.googleusercontent.com/a/AATXAJ...,Not bad,5,0,23.9.5,2022-04-05 23:17:34,NaN,NaN
3,gp:AOqpTOFHDm-Qa5R6jCpOGTFT2qr1_PKbCTbBNPahCEn...,Keli We,https://play-lh.googleusercontent.com/a-/AOh14...,It is good,2,0,22.2.5,2022-04-05 23:17:04,NaN,NaN
4,gp:AOqpTOFB6Ndao8IHRpOJRmbSknwMGxHcwYzux93YyXI...,Mavis Kotoka,https://play-lh.googleusercontent.com/a/AATXAJ...,Very interesting app,5,0,22.1.5,2022-04-05 23:17:04,NaN,NaN


In [4]:
df.shape

(307057, 10)

In [5]:
df.isnull().sum()

reviewId                     0
userName                     0
userImage                    0
content                      4
score                        0
thumbsUpCount                0
reviewCreatedVersion     89216
at                           0
replyContent            306938
repliedAt               306938
dtype: int64

### Since review id, user name and user Image does not content any valuable information, we can simply drop those columns:

In [6]:
df.drop(['reviewId','userName','userImage'], axis = 1, inplace = True)
df.head()

,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,Good,5,0,23.8.4,2022-04-05 23:18:30,NaN,NaN
1,Awesome app! Too many people on it where it's ...,5,0,NaN,2022-04-05 23:18:21,NaN,NaN
2,Not bad,5,0,23.9.5,2022-04-05 23:17:34,NaN,NaN
3,It is good,2,0,22.2.5,2022-04-05 23:17:04,NaN,NaN
4,Very interesting app,5,0,22.1.5,2022-04-05 23:17:04,NaN,NaN


### Since reviewCreatedVersion, at, replyContent and repliedAt have many missing values, we can simply drop those columns:

In [7]:
df.drop(['reviewCreatedVersion','at','replyContent','repliedAt'], axis = 1, inplace = True)
df.head()

,content,score,thumbsUpCount
0,Good,5,0
1,Awesome app! Too many people on it where it's ...,5,0
2,Not bad,5,0
3,It is good,2,0
4,Very interesting app,5,0


# Performing Text Data Analysis:

### Importing Libraries:

In [8]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

In [9]:
lemma = WordNetLemmatizer()
stop_words = stopwords.words('english')

### Creating a function to clear the text:

In [10]:
def text_prep(x):
     corp = str(x).lower() 
     corp = re.sub('[^a-zA-Z]+',' ', corp).strip() 
     tokens = word_tokenize(corp)
     words = [t for t in tokens if t not in stop_words]
     lemmatize = [lemma.lemmatize(w) for w in words]
     return lemmatize

### Apply this function to every text in the 'content‘ column:

In [12]:
preprocess_tag = [text_prep(i) for i in df['content']]
df["preprocess_txt"] = preprocess_tag

### Calculate the number of resultant words for each text:

In [13]:
df['word_count'] = df['preprocess_txt'].map(lambda x: len(x)) ## word_count

### Importing Neagtive and Positive texts

In [14]:
file = open('E:/Data Science/Data Sets/TikTok - Google Play Store/Positive and Negative words/negative-words.txt', 'r')
neg_words = file.read().split()
file = open('E:/Data Science/Data Sets/TikTok - Google Play Store/Positive and Negative words/positive-words.txt', 'r')
pos_words = file.read().split()

## Lets calculate Positive and Negative score of each comment:

In [15]:
num_pos = df['preprocess_txt'].map(lambda x: len([i for i in x if i in pos_words]))
df['pos_count'] = num_pos
num_neg = df['preprocess_txt'].map(lambda x: len([i for i in x if i in neg_words]))
df['neg_count'] = num_neg

### Calculating Polarity Score and Subjectivity Score:

In [16]:
df['Polarity_Score'] = round((df['pos_count'] - df['neg_count']) / ((df['pos_count'] + df['neg_count']) + 0.00001), 2)
df['Subjectivity_Score'] = round((df['pos_count'] + df['neg_count']) / (df['word_count'] + 0.00001), 2)

# 2) Analysis of Readability:

### Creating a function to clear the text without lemmatizing:

In [17]:
def text_prep_2(x):
    corp = str(x)
    corp = re.sub('[^a-zA-Z]+',' ', corp).strip() 
    tokens = word_tokenize(corp)
    return tokens

### Apply this function to every text in the 'content‘ column

In [18]:
preprocess_tokens = [text_prep_2(i) for i in df['content']]
df["preprocess_tokens"] = preprocess_tokens

### Calculate the number of resultant words for each text:

In [19]:
df['total_token_len'] = df['preprocess_tokens'].map(lambda x: len(x))

### Creating a function to clear the sentences:

In [25]:
def text_prep_2(x):
    corp = str(x)
    corp = re.sub('[^a-zA-Z]+',' ', corp).strip() 
    sentences_all = nltk.tokenize.sent_tokenize(corp)
    return sentences_all

### Apply this function to every text in the 'content‘ column

In [27]:
preprocess_sentences = [text_prep_2(i) for i in df['content']]
df["preprocess_sentences_all"] = preprocess_sentences

### Calculating Average Sentence Length:

In [29]:
total_sentence_count = []
avg_sentence_lenght_count = []
for i in range(0,df['content'].shape[0]):
    temp = len(df['preprocess_sentences_all'][i])
    total_sentence_count.append(temp)
    temp_2 = round(df['total_token_len'][i]/temp , 2)
    avg_sentence_lenght_count.append(temp_2)
df['Average_Sentence_Length'] = avg_sentence_lenght_count

C:\Users\vipul\AppData\Local\Temp/ipykernel_2920/2953880600.py:6: RuntimeWarning: invalid value encountered in longlong_scalars
  temp_2 = round(df['total_token_len'][i]/temp , 2)


## Syllable count per word:

In [30]:
def word_counter(word):
    count=0
    for w in word:
        count = count + 1
    return count

In [31]:
def syllable_counter(word):
    syllable_count=0
    for w in word:
        if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u' or w=='A' or w=='E' or w=='I' or w=='O' or w=='U'):
            syllable_count=syllable_count+1
    if word.endswith("ed"):
        syllable_count=syllable_count-1
    if word.endswith("es"):
        syllable_count=syllable_count-1
    return syllable_count

In [39]:
syllable_count_counter = []
for i in range(0,df['content'].shape[0]):
    temp = round((sum((syllable_counter(word)/word_counter(word))+0.001 for word in df['preprocess_tokens'][i]) / (len(df['preprocess_tokens'][i])+0.001)),2)
    syllable_count_counter.append(temp)
df['syllable_count'] = syllable_count_counter

## Average Number of Words Per Sentence:

In [40]:
df['Avg_no_words_per_sentence'] = df['Average_Sentence_Length']

## Complex word count:

In [41]:
complex_words = []
for i in range(0,df['content'].shape[0]):
    complex_word_counter = 0
    for word in df['preprocess_tokens'][i]:
        temp = syllable_counter(word)
        if temp>2:
            complex_word_counter = complex_word_counter + 1
    complex_words.append(complex_word_counter)
df['complex_word_count'] = complex_words

## Percentage of complex words

In [43]:
Percentage_Complex_words_count = []
for i in range(0,df['content'].shape[0]):
    temp = round((df['complex_word_count'][i]/df['total_token_len'][i])*100,2)
    Percentage_Complex_words_count.append(temp)
df['Percentage_of_Complex_words'] = Percentage_Complex_words_count

C:\Users\vipul\AppData\Local\Temp/ipykernel_2920/3838803951.py:3: RuntimeWarning: invalid value encountered in longlong_scalars
  temp = round((df['complex_word_count'][i]/df['total_token_len'][i])*100,2)


## Fog Index:

In [45]:
fog_index_count = []
for i in range(0,df['content'].shape[0]):
    temp = round((df['Average_Sentence_Length'][i] + df['Percentage_of_Complex_words'][i])*0.4,2)
    fog_index_count.append(temp)
df['Fog_Index'] = fog_index_count

## Average Word Lenght calculation:

In [50]:
def avg_word_lenght(words):
    average_word_lenght = round(sum(len(word) for word in words) / (len(words)+0.001),2)
    return average_word_lenght

In [51]:
avg_word_lenght_counter = []
for i in range(0,df['content'].shape[0]):
    temp = avg_word_lenght(df['preprocess_tokens'][i])
    avg_word_lenght_counter.append(temp)
df['Avg_word_lenght'] = avg_word_lenght_counter

### Removing unnecessary Columns:

In [52]:
df.columns.values

array(['content', 'score', 'thumbsUpCount', 'preprocess_txt',
       'word_count', 'pos_count', 'neg_count', 'Polarity_Score',
       'Subjectivity_Score', 'preprocess_tokens', 'total_token_len',
       'preprocess_sentences_all', 'Average_Sentence_Length',
       'syllable_count', 'Avg_no_words_per_sentence',
       'complex_word_count', 'Percentage_of_Complex_words', 'Fog_Index',
       'Avg_word_lenght'], dtype=object)

In [53]:
df.drop(['preprocess_sentences_all', 'preprocess_txt','preprocess_tokens', 'total_token_len'],axis=1,inplace=True)

## Reordering the columns:

In [56]:
column_names = ['content', 'score', 'thumbsUpCount', 'pos_count', 'neg_count',
       'Polarity_Score', 'Subjectivity_Score','Average_Sentence_Length',
       'Percentage_of_Complex_words','Fog_Index','Avg_no_words_per_sentence',
        'complex_word_count','word_count', 'Avg_word_lenght']

In [57]:
df = df[column_names]
df.head()

,content,score,thumbsUpCount,pos_count,neg_count,Polarity_Score,Subjectivity_Score,Average_Sentence_Length,Percentage_of_Complex_words,Fog_Index,Avg_no_words_per_sentence,complex_word_count,word_count,Avg_word_lenght
0,Good,5,0,1,0,1.00,1.00,1.0,0.00,0.40,1.0,0,1,4.00
1,Awesome app! Too many people on it where it's ...,5,0,6,1,0.71,0.47,30.0,13.33,17.33,30.0,4,15,3.63
2,Not bad,5,0,0,1,-1.00,1.00,2.0,0.00,0.80,2.0,0,1,3.00
3,It is good,2,0,1,0,1.00,1.00,3.0,0.00,1.20,3.0,0,1,2.67
4,Very interesting app,5,0,1,0,1.00,0.50,3.0,33.33,14.53,3.0,1,2,6.00


# Saving the DataFrame as XLSX file:

In [59]:
# determining the name of the file
file_name = 'E:/Data Science/Data Sets/TikTok - Google Play Store/Output_Data_final.xlsx'
  
# saving the excel
df.to_excel(file_name)
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.
